## 10.2 Keras介绍
Keras是目前使用最为广泛的深度学习工具之一，它的底层可以支持TensorFlow、MXNet、CNTK和Theano。如今，Keras更是直接被引入TensorFlow的核心代码库，成为TensorFlow官方提供的高层封装之一。

### 10.2.1 Keras基本用法
**和TFLearn API类似，Keras API也对模型定义、损失定义、训练过程等进行了封装，而且封装之后的整个训练过程和TFLearn也基本一致，可以分为数据处理、模型定义、模型训练三个部分**。Keras安装：`pip install keras`。

**1. Keras-CNN**

以下代码展示了如何使用原生态Keras在MNIST数据集上实现LeNet-5模型：

In [1]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras import backend as K

# 1. 数据预处理
num_classes = 10
img_rows, img_cols = 28, 28
 
# 通过Keras封装好的API加载MNIST数据。其中trainX就是一个60000 * 28 * 28的数组，
# trainY是每一张图片对应的数字。
(trainX, trainY), (testX, testY) = mnist.load_data()

# 因为不同的底层（TensorFlow或MXNet）对输入的要求不一样，所以这里需要
# 根据对图像编码的格式要求来设置输入层的格式。
if K.image_data_format() == 'channels_first':
    trainX = trainX.reshape(trainX.shape[0], 1, img_rows, img_cols)
    testX = testX.reshape(testX.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    trainX = trainX.reshape(trainX.shape[0], img_rows, img_cols, 1)
    testX = testX.reshape(testX.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
    
# 将图像像素转化为0到1之间的实数
trainX = trainX.astype('float32')
testX = testX.astype('float32')
trainX /= 255.0
testX /= 255.0
 
# 将标准答案转化为需要的格式（one-hot编码）。
trainY = keras.utils.to_categorical(trainY, num_classes)
testY = keras.utils.to_categorical(testY, num_classes)


# 2. 使用Keras API定义模型。
model = Sequential()

# 一层深度为32，过滤器大小为5*5的卷积层
model.add(Conv2D(32, kernel_size=(5, 5), activation='relu', input_shape=input_shape))
# 一层过滤器大小为2*2的最大池化层
model.add(MaxPooling2D(pool_size=(2, 2)))
# 一层深度为64，过滤器大小为5*5的卷积层
model.add(Conv2D(64, (5, 5), activation='relu'))
# 一层过滤器大小为2*2的最大池化层
model.add(MaxPooling2D(pool_size=(2, 2)))
# 将卷积层的输出拉直后作为下面全连接层的输入
model.add(Flatten())
# 全连接层，由500个节点
model.add(Dense(500, activation='relu'))
# 全连接层，得到最后的输出
model.add(Dense(num_classes, activation='softmax'))
 
# 定义损失函数、优化函数和评测方法。
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.SGD(),
              metrics=['accuracy'])


# 3. 通过Keras的API训练模型并计算在测试数据上的准确率
model.fit(trainX, trainY,
          batch_size=128,
          epochs=10,
          validation_data=(testX, testY))
 
# 在测试数据上计算准确率。
score = model.evaluate(testX, testY)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Using TensorFlow backend.


Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 9s 152us/step - loss: 0.9914 - acc: 0.7421 - val_loss: 0.3020 - val_acc: 0.9108
Epoch 2/10
60000/60000 [==============================] - 3s 49us/step - loss: 0.2580 - acc: 0.9227 - val_loss: 0.1868 - val_acc: 0.9460
Epoch 3/10
60000/60000 [==============================] - 3s 49us/step - loss: 0.1852 - acc: 0.9446 - val_loss: 0.1452 - val_acc: 0.9583
Epoch 4/10
60000/60000 [==============================] - 3s 49us/step - loss: 0.1464 - acc: 0.9566 - val_loss: 0.1183 - val_acc: 0.9646
Epoch 5/10
60000/60000 [==============================] - 3s 49us/step - loss: 0.1222 - acc: 0.9637 - val_loss: 0.1052 - val_acc: 0.9692
Epoch 6/10
60000/60000 [==============================] - 3s 48us/step - loss: 0.1052 - acc: 0.9689 - val_loss: 0.0927 - val_acc: 0.9723
Epoch 7/10
60000/60000 [==============================] - 3s 48us/step - loss: 0.0940 - acc: 0.9724 - val_loss: 0.0834 - val_ac

从以上代码可以看出使用Keras API训练模型过程为：
1. **先定义一个Sequential类**；
2. **然后在Sequential实例中通过add函数添加网络层**，Keras把卷积层、池化层、RNN结构（LSTM、GRN）、全连接层等常用的神经网络结构都做了封装，可以很方便地实现深层神经网络；
3. **再通过compile函数指定优化函数、损失函数以及训练过程中需要监控的指标等**，Keras对优化函数、损失函数以及监控指标都有封装，同时也支持使用自定义的方式，在[Keras的API文档](https://keras.io/)中有详细的介绍，这里不再赘述；
4. **最后Sequential实例可以通过fit函数来训练模型**，类似TFLearn中的fit函数， Keras的fit函数只须给出训练数据、batch大小和训练轮数，Keras就可以自动完成模型训练的整个过程。

**2. Keras-RNN**

除了能够很方便地处理图像问题，Keras对于循环神经网络的支持也是非常出色的。有了Keras API，循环神经网络的循环体结构也可以通过简单的一句命令完成。

使用循环神经网络判断语言的情感（比如在以下例子中需要判断的一个评价是好评还是差评）和自然语言建模问题类似，唯一的区别在于除了最后一个时间点的输出是有意义的，其他时间点的输出都可以忽略。图10.1展示了使用循环神经网络处理情感分析问题的模型结构。
<p align='center'>
    <img src=images/图10.1.JPG>
</p>

以下代码给出了如何通过Keras实现自然语言情感分类问题。

In [1]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb

# 1. 数据预处理
max_features = 20000     # 最多使用的单词数
maxlen = 80              # 循环网络的截断长度
batch_size = 32

# 加载数据并将单词转化为ID。和自然语言模型类似，会将出现频率较低的单词替换为统一的ID。
# 通过keras封装的API会生成25000条训练数据和25000条测试数据，
# 每一条数据可以被看成一段话，并且每段话都有一个好评或者差评的标签。
(trainX, trainY), (testX, testY) = imdb.load_data(num_words=max_features)
print(len(trainX), 'train sequences')
print(len(testX), 'test sequences')

# 在自然语言中，每一段话的长度是不一样的，但循环神经网络的循环长度是固定的，
# 所以这里需要先将所有段落统一成固定长度。不够的填充0，超过的忽略超多的部分。
trainX = sequence.pad_sequences(trainX, maxlen=maxlen)
testX = sequence.pad_sequences(testX, maxlen=maxlen)
print('trainX shape:', trainX.shape)
print('testX shape:', testX.shape)


# 2. 模型定义
model = Sequential()

# 构建Embedding层，128代表了embedding向量的维度。
model.add(Embedding(max_features, 128))
# 构建LSTM层。
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
# 构建最后的全连接层，注意上面在构建的LSTM层时只会得到最后一个节点的输出。
# 如果需要输出每个节点的结果，可以将return_sequence设置为True
model.add(Dense(1, activation='sigmoid'))

# 与MNIST样例类似地指定损失函数、优化方法和评估指标。
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


# 3. 模型训练、测试
# 与MNIST样例类似地指定训练数据、训练轮数、batch大小以及验证数据。
model.fit(trainX, trainY,
          batch_size=batch_size,
          epochs=10,
          validation_data=(testX, testY))

score = model.evaluate(testX, testY, batch_size=batch_size)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Using TensorFlow backend.


17465344/17464789 [==============================] - 66s 4us/step
25000 train sequences
25000 test sequences
trainX shape: (25000, 80)
testX shape: (25000, 80)
Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 127s 5ms/step - loss: 0.4603 - acc: 0.7836 - val_loss: 0.3869 - val_acc: 0.8271
Epoch 2/10
25000/25000 [==============================] - 126s 5ms/step - loss: 0.3004 - acc: 0.8767 - val_loss: 0.3889 - val_acc: 0.8284
Epoch 3/10
25000/25000 [==============================] - 125s 5ms/step - loss: 0.2173 - acc: 0.9156 - val_loss: 0.4181 - val_acc: 0.8278
Epoch 4/10
25000/25000 [==============================] - 124s 5ms/step - loss: 0.1556 - acc: 0.9422 - val_loss: 0.4546 - val_acc: 0.8213
Epoch 5/10
25000/25000 [==============================] - 122s 5ms/step - loss: 0.1096 - acc: 0.9600 - val_loss: 0.5533 - val_acc: 0.8236
Epoch 6/10
25000/25000 [==============================] - 122s 5ms/step - loss: 0.0813 - acc: 0.9707

### 10.2.2 keras高级用法
10.2.1小节对Keras基本用法进行了详细介绍，虽然通过Keras地封装，很多经典地神经网络结构可以很快地被实现，不过要实现一些更加灵活地网络结构、损失函数或者数据输入方法，就需要对Keras地高级用法有更多的了解。

**1. 返回值形式定义网络（函数式）**

上小节中一个最重要地封装就是Sequential类，所有神经网络模型定义和训练都是通过这个类实现的。然而从其名字就可以看出，**Sequential只支持顺序模型地定义**。类似Inception这样的模型结构，通过Sequential类就不容易直接实现了，**为了支持更加灵活的模型定义方法，Keras支持以返回值的形式定义网络层结构**。可以参考[Keras官网文档对这两种构建网络的介绍](https://keras.io/models/about-keras-models/)。下面代码展示了如何使用这种方式定义模型：

In [2]:
# 对比第一个的cell

import keras
from tflearn.layers.core import fully_connected
from keras.datasets import mnist
from keras.layers import Input, Dense
from keras.models import Model

# 1. 数据预处理
num_classes = 10
img_rows, img_cols = 28, 28
 
# 通过Keras封装好的API加载MNIST数据。
# 对比10.2.1中keras-CNN，由于这里使用全连接层，不需要将输入整理成三维矩阵
(trainX, trainY), (testX, testY) = mnist.load_data()
trainX = trainX.reshape(trainX.shape[0], img_rows * img_cols)
testX = testX.reshape(testX.shape[0], img_rows * img_cols)
 
# 将图像像素转化为0到1之间的实数。
trainX = trainX.astype('float32')
testX = testX.astype('float32')
trainX /= 255.0
testX /= 255.0
 
# 将标准答案转化为需要的格式（one-hot编码）。
trainY = keras.utils.to_categorical(trainY, num_classes)
testY = keras.utils.to_categorical(testY, num_classes)


# 2. 通过返回值的方式定义模型
inputs = Input(shape=(784,))

x = Dense(500, activation='relu')(inputs)
predictions = Dense(10, activation='softmax')(x)

model = Model(inputs=inputs, outputs=predictions)
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.SGD(),
              metrics=['accuracy'])


# 3. 训练模型
model.fit(trainX, trainY,
          batch_size=32,
          epochs=10,
          validation_data=(testX, testY))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 9s 150us/step - loss: 0.6081 - acc: 0.8498 - val_loss: 0.3436 - val_acc: 0.9068
Epoch 2/10
60000/60000 [==============================] - 5s 78us/step - loss: 0.3235 - acc: 0.9107 - val_loss: 0.2790 - val_acc: 0.9220
Epoch 3/10
60000/60000 [==============================] - 5s 78us/step - loss: 0.2751 - acc: 0.9229 - val_loss: 0.2472 - val_acc: 0.9318
Epoch 4/10
60000/60000 [==============================] - 5s 78us/step - loss: 0.2444 - acc: 0.9325 - val_loss: 0.2230 - val_acc: 0.9386
Epoch 5/10
60000/60000 [==============================] - 5s 78us/step - loss: 0.2212 - acc: 0.9389 - val_loss: 0.2074 - val_acc: 0.9419
Epoch 6/10
60000/60000 [==============================] - 5s 80us/step - loss: 0.2024 - acc: 0.9441 - val_loss: 0.1913 - val_acc: 0.9470
Epoch 7/10
60000/60000 [==============================] - 5s 81us/step - loss: 0.1868 - acc: 0.9480 - val_loss: 0.1797 - val_ac

通过这样的方式，Keras就可以实现类似Inception这样的模型结构，以下代码展示了如何通过Keras实现Inception结构：

In [ ]:
'''
from keras.layers import Conv2D, MaxPooling2D, Input

# 定义输入图像的尺寸
input_img = Input(shape=(256, 256, 3))

# 定义第一个分支
tower_1 = Conv2D(64, (1, 1), padding='same', activation='relu')(input_img)
tower_1 = Conv2D(64, (3, 3), padding='same', activation='relu')(tower_1)

# 定义第二个分支。与顺序模型不同，第二个分支的输入时input_img，而不是第一分支的输出
tower_2 = Conv2D(64, (1, 1), padding='same', activation='relu')(input_img)
tower_2 = Conv2D(64, (5, 5), padding='same', activation='relu')(tower_2)

# 定义第三个分支。类似的，输入也是input_img
tower_3 = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(input_img)
tower_3 = Conv2D(64, (1, 1), padding='same', activation='relu')(tower_3)

# 将三个分支通过concatenate的方式拼接在一起
output = keras.layers.concatenate([tower_1, tower_2, tower_3], axis=1)
'''

**2. 多输入输出**

除了可以支持非顺序模型，Keras也可以支持有多个输入或输出的模型，下图是一种多输入输出的网络结构：
<p align='center'>
    <img src=images/图10.2.JPG>
</p>

- 输入层1含有784个节点，代表MNIST图片中的784个像素；
- 输入层2含有10个节点，代表该图片所对应的数字；
- 输出层1在预测时仅依赖维度为1的隐藏层，因此预测的准确度较低；
- 输出层2中直接包含了正确答案，因此准确率较高。

下面代码给出了如何实现上图的网络结构：

In [3]:
import keras
from tflearn.layers.core import fully_connected
from keras.datasets import mnist
from keras.layers import Input, Dense
from keras.models import Model

# 1. 数据预处理
num_classes = 10
img_rows, img_cols = 28, 28
 
# 通过Keras封装好的API加载MNIST数据。
(trainX, trainY), (testX, testY) = mnist.load_data()
trainX = trainX.reshape(trainX.shape[0], img_rows * img_cols)
testX = testX.reshape(testX.shape[0], img_rows * img_cols)

trainX = trainX.astype('float32')
testX = testX.astype('float32')
trainX /= 255.0
testX /= 255.0

trainY = keras.utils.to_categorical(trainY, num_classes)
testY = keras.utils.to_categorical(testY, num_classes)


# 2. 定义模型
# 定义两个输入。
input1 = Input(shape=(784,), name = "input1")
input2 = Input(shape=(10,), name = "input2")

# 定义第一个输出。
x = Dense(1, activation='relu')(input1)
output1 = Dense(10, activation='softmax', name = "output1")(x)

# 定义第二个输出。
y = keras.layers.concatenate([x, input2])
output2 = Dense(10, activation='softmax', name = "output2")(y)

# 定义一个有多个输入和输出的模型，这里需要将所有的输入和输出给出即可
model = Model(inputs=[input1, input2], outputs=[output1, output2])

# 定义损失函数、优化函数和评测方法。
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.SGD(),
              loss_weights = [1, 0.1],
              metrics=['accuracy'])


# 3. 训练模型
model.fit([trainX, trainY], [trainY, trainY],
          batch_size=128,
          epochs=20,
          validation_data=([testX, testY], [testY, testY]))

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 6s 102us/step - loss: 2.4584 - output1_loss: 2.2186 - output2_loss: 2.3984 - output1_acc: 0.1795 - output2_acc: 0.1022 - val_loss: 2.4028 - val_output1_loss: 2.1664 - val_output2_loss: 2.3639 - val_output1_acc: 0.1963 - val_output2_acc: 0.1010
Epoch 2/20
60000/60000 [==============================] - 2s 30us/step - loss: 2.3731 - output1_loss: 2.1404 - output2_loss: 2.3272 - output1_acc: 0.1973 - output2_acc: 0.1022 - val_loss: 2.3367 - val_output1_loss: 2.1075 - val_output2_loss: 2.2922 - val_output1_acc: 0.1944 - val_output2_acc: 0.1010
Epoch 3/20
60000/60000 [==============================] - 2s 29us/step - loss: 2.3146 - output1_loss: 2.0900 - output2_loss: 2.2459 - output1_acc: 0.2088 - output2_acc: 0.1719 - val_loss: 2.2830 - val_output1_loss: 2.0628 - val_output2_loss: 2.2018 - val_output1_acc: 0.2272 - val_output2_acc: 0.1932
Epoch 4/20
60000/60000 [======================

对于多输入输出需要注意的是：
- **对于损失函数**：若多个输出的损失函数相间，可以只指定一个损失函数；若多个输出的损失函数不同，则可以通过一个列表或一个字典来指定每一个输出的损失函数，比如可以使用：`loss={'output1':'binary_crossentropy', 'output2':'binary_crossentropy'｝`来为不同的输出指定不同的损失函数。

- **不同损失权重**：类似地， Keras也支持为不同输出产生的损失指定权重，这可以通过loss_weights参数来完成。比如在上例的定义中，output1的权重为1,output2的权重为0.1，所以这个模型会更加偏向于优化第一个输出。

- **训练的输入**：在模型训练过程，因为有两个输入和输出，所以这里提供的数据也需要有两个输入和两个期待的正确答案输出。通过列表的方式提供数据时， Keras会假设数据给出的顺序和定义Model类时输入输出给出的顺序是对应的。为了避免顺序不一致导致的问题，本书更推荐使用字典的形式给出：

`model.fit(
  {'input1': trainX,'input2': trainY},
  {'output1': trainY,'output2': trainY},
   ...)
`

从以上输出可以看出Keras在训练过程中会显示每个输出层的loss和accuracy。因为输出层output1只使用了一个维度为1的隐藏节点，所以正确率只有29.85%。虽然输出层output2使用了正确答案作为输入，但是因为在损失函数中权重较低（只有0.1），所以它的收敛速度较慢，在20个epoch 时准确率也只有86.3%。如果将两个输出层的损失权重设为一样，那么输出层output1在20个epoch时的准确率将只有27%，而输出层output2的准确率可以达到99.9%。

**3. Keras + 原生态TensorFlow API**

虽然通过返回值的方式已经可以实现大部分的神经网络模型，然而Keras API还存在两大问题：
- 第一，原生态Keras API对训练数据的处理流程支持得不太好，基本上需要一次性将数据全部加载到内存；
- 第二，原生态Keras API无法支持分布式训练。

为了解决这两个问题，Keras提供了一种与原生态TensorFlow结合得更加紧密的方式。以下代码显示了如何联合二者来解决MNIST问题：

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist_data = input_data.read_data_sets('../../datasets/MNIST_data', one_hot=True)

# 通过TensorFlow中的placeholder定义输入。类似地，Keras封装的网络层结构也可以支持
# 使用第7章中介绍的输入队列。这样可以有效避免一次性加载所有数据的问题。
x = tf.placeholder(tf.float32, shape=(None, 784))
y_ = tf.placeholder(tf.float32, shape=(None, 10))

# 直接使用TensorFlow中提供的Keras API定义网络层结构
net = tf.keras.layers.Dense(500, activation='relu')(x)
y = tf.keras.layers.Dense(10, activation='softmax')(net)

# 定义损失函数和优化方法。注意这里可以混用Keras的API和原生态TensorFlow的API
loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_, y))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

# 定义预测正确率作为指标
acc_value = tf.reduce_mean(tf.keras.metrics.categorical_accuracy(y_, y))

# 使用原生态TensorFlow的方式训练模型，这样可以有效实现分布式。
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for i in range(10000):
        xs, ys = mnist_data.train.next_batch(100)
        _, loss_value = sess.run([train_step, loss], feed_dict={x: xs, y_: ys})
        if i % 1000 == 0:
            print("After %d training step(s), loss on training batch is "
                  "%g." % (i, loss_value))

    test_acc = acc_value.eval(feed_dict={x: mnist_data.test.images,
                                    y_: mnist_data.test.labels})
    print('Final test accuracy is %f.' % test_acc)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../../datasets/MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../../datasets/MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ../../datasets/MNIST_data\t10k-images-idx3-ubyte.gz
Extracting ../../datasets/MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
After 0 training step(s), loss on training batch is 2.37641.
After 1000 training step(s), loss on training batch is 0.0315625.
After 2000 training step(s), loss on training batch is 0.0420956.
After 3000 training step(s), loss on training ba

通过Keras和原生态TensorFlow更紧密地结合，可以使建模的灵活性进一步提高，但是同时也会损失一部分封装带来的易用性。所以在实际问题中读者可以根据需求合理地选择封装的程度。